# Co-locating Temperature Anomaly Values between CBP Monitoring Stations and Satellite SST

## Notebook Overview
### Tasks
- Select satellite SST and SST anomaly values for each location and date in which there is an in situ observation from the Chesapeake Bay Program
- Save two csvs of the results: one csv for in situ and satellite SSTs and a second csv for the in situ and satellite SST anomalies

This script creates a csv which holds the corresponding validation data points from both Chesapeake Bay Program's (CBP) Water Quality dataset and the two satellite datasets. The csv contains one row for each day and pixel in which there was a CBP validation point, and either a MUR or a Geopolar satellite pixel.
Output: csv of temperature values that can be observed in the satellite SST datasets and the in situ CBP data

## Analysis

In [1]:
import os
from pathlib import Path
import warnings

import xarray as xr
import pandas as pd
import numpy as np

In [2]:
REPO_ROOT = Path('/Users/rwegener/repos/chesapeake_mhw')
SAVE_FIGS = True

## Read CBay Program In Situ Anomaly Data

In [3]:
# path_anom = REPO_ROOT / 'data/interim' / 'cbp_temps_selected_stations_with_climatology.csv'
path_anom = REPO_ROOT / 'data/02_interim' / 'cbp_ssta_selected_stations_with_climatology.csv'
anom_raw = pd.read_csv(path_anom, parse_dates=[1])

In [4]:
# path_sst = REPO_ROOT / 'data/interim' / 'cbp_stations_climatology_raw_filtered.csv'
path_sst = REPO_ROOT / 'data/02_interim' / 'cbp_sst_depthaveraged.csv'
sst_raw = pd.read_csv(path_sst, parse_dates=[1])

In [5]:
# anom_dates = pd.read_csv(
#     REPO_ROOT / 'data/interim' / 'climatology_station_dates.csv',
#     parse_dates=[1],
# )

## Helper Functions

Creating functions for repeated tasks

In [6]:
def extract_matching_value(matching_array):
    if matching_array.ndim == 0:
        matching_sst = matching_array
    elif matching_array.ndim == 1 and matching_array.size == 1:
        matching_sst = matching_array[0]
    else:
        matching_sst = np.nan
    return matching_sst

In [7]:
def get_satellite_sst(full_sst, lat, lon, time):
    '''
    For a given latitude, longitude, and time extract the SST value at that location
    in the give satellite dataset.
    :full_sst: 3D array of satellite SST from which to extract temperature values.
    Array must have dimensions called "lat", "lon", and "time".
    :lat: latitude value from the in situ dataset used to find the nearest SST pixel
    :lon: longitude value from the in situ dataset used to find the nearest SST pixel
    :time: time value from the in situ dataset to match in the SST dataset
    '''
    try:
        # time does NOT have nearest interpolation because we do not want adjacent days to
        # be selected
        time = time.strftime('%Y-%m-%d')
        matching_array = full_sst.sel(lat=lat, lon=lon, 
                                      method='nearest').sel(time=time).values
        # Check only one value is returned, allowing for multiple array size 
        # return shapes
        matching_sst = extract_matching_value(matching_array)
    except KeyError:
        # If a key error was raised the corresponding date was not found. 
        # Return nan for that sample location
        print('In situ date not found in satellite SST', time, 'at',
            lat, lon)
        matching_sst = np.nan
    return matching_sst

## Initialize the output dataframe

In [9]:
anom_raw = anom_raw[['Station', 'Latitude', 'Longitude', 'SampleDate', 'anom_cbp']]
# wq_anom = anom_raw.copy()
# wq_anom['anom_geopolar'] = -999
# wq_anom['anom_mur'] = -999
# wq_anom['anom_ostia'] = -999

In [10]:
sst_raw = sst_raw[['Station', 'Latitude', 'Longitude', 'SampleDate', 'sst_cbp']]
wq_sst = sst_raw.copy()
wq_sst['geopolar'] = -999
wq_sst['mur'] = -999
wq_sst['ostia'] = -999

## Extract SST value corresponding to CBP In situ observations

### Geo-Polar SST

In [11]:
# Open raw SST
path = (
    REPO_ROOT / 'data/01_raw' / 
    'L4_GHRSST-SSTfnd-Geo_Polar_Blended_Night-GLOB-v02.0-fv01.0_CB_20030101_20231231.nc'
)
geopolar = xr.open_dataset(path).analysed_sst
# convert kelvin to celsius & update metadata
geopolar.values = geopolar.values - 273.15
geopolar.attrs.update({'units': 'celsius',})

# Open calculated climatological SST
path = REPO_ROOT / 'data/02_interim' / 'geopolar_climatology_chesapeake.nc'
geopolar_clim = xr.open_dataset(path).climatology

#### Compute the custom climatology

Create monthly climatologies using only the dates which have corresponding observations from the CBP data.

In [12]:
anom_raw = anom_raw.set_index(['Station', 'SampleDate'])

In [13]:
%%time

satellite = 'geopolar'

anom_raw[f'clim_{satellite}'] = -999
anom_raw[f'sst_{satellite}'] = -999
for station in anom_raw.index.get_level_values(0).unique():
    print('station:', station)
    station_df = anom_raw.loc[station]
    # latitude and longitude should be the same for each row, so pick 0th value
    lat = station_df.Latitude[0]
    lon = station_df.Longitude[0]
    # loop through months
    for month in range(1, 13):
        # compile all dates from that month
        month_obs_dates = station_df[station_df.index.month == month].index
        # extract corresponding satellite values
        month_obs = []
        for date in month_obs_dates:
            sst = geopolar.sel(lat=lat, lon=lon, method='nearest') \
                .sel(time=date.strftime('%Y-%m-%d')).values
            sst = extract_matching_value(sst)
            # save sst observation value in the dataframe
            anom_raw.loc[(station, date), f'sst_{satellite}'] = sst
            # append value to array for climatology
            month_obs.append(sst)
        # average to make mean (--> ex. january climatological value)
        mean = np.nanmean(np.array(month_obs))
        # save sst_clim to a new row (subtract later)
        for date in month_obs_dates:
            anom_raw.loc[(station, date), f'clim_{satellite}'] = mean

station: CB1.1
station: CB2.1
station: CB2.2
station: CB3.1
station: CB3.2
station: CB3.3C
station: CB4.1C
station: CB4.2C
station: CB4.3C
station: CB4.4
station: CB5.1
station: CB5.2
station: CB5.3
station: CB6.2
station: CB6.3
station: CB6.4
station: CB7.3
station: CB7.3E
station: CB7.4
station: CB7.4N
station: CB8.1
station: CB8.1E
station: EE1.1
station: EE2.1
station: EE2.2
station: EE3.0
station: EE3.1
station: EE3.2
station: ET2.2
station: ET2.3


<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice


station: ET4.2
station: ET5.2
station: ET6.2
station: ET8.1
station: ET9.1
station: LE2.2
station: LE2.3
station: LE3.6
station: LE5.5-W
station: RET2.2
station: RET2.4
station: WE4.1
station: WE4.2
station: WE4.3
station: WE4.4
station: WT3.1
station: WT5.1


<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty

station: WT6.1
station: WT8.1
station: WT8.2
station: WT8.3
CPU times: user 30.3 s, sys: 763 ms, total: 31 s
Wall time: 31.2 s


<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice


In [14]:
# Calculate anomaly
anom_raw['anom_geopolar'] = anom_raw['sst_geopolar'] - anom_raw['clim_geopolar']

In [15]:
anom_raw

Latitude  Longitude  anom_cbp  clim_geopolar  \
Station SampleDate                                                 
CB1.1   2003-01-15  39.54794  -76.08481    -2.950       3.399167   
        2005-01-13  39.54794  -76.08481     1.450       3.399167   
        2006-01-12  39.54794  -76.08481     0.950       3.399167   
        2007-01-12  39.54794  -76.08481     1.150       3.399167   
        2008-01-17  39.54794  -76.08481     0.650       3.399167   
...                      ...        ...       ...            ...   
WT8.3   2019-02-11  38.84250  -76.53410    -0.155       2.933501   
        2020-02-13  38.84250  -76.53410     2.945       2.933501   
        2021-02-08  38.84250  -76.53410    -0.755       2.933501   
        2022-02-09  38.84250  -76.53410    -0.905       2.933501   
        2023-02-16  38.84250  -76.53410     3.445       2.933501   

                    sst_geopolar  anom_geopolar  
Station SampleDate                               
CB1.1   2003-01-15      1.630005      -1.769162  
        2005-01-13      3.160004      -0.239164  
        2006-01-12      3.910004       0.510836  
        2007-01-12      4.630005       1.230838  
        2008-01-17      3.709991       0.310824  
...                          ...            ...  
WT8.3   2019-02-11      2.260010      -0.673491  
        2020-02-13      3.980011       1.046510  
        2021-02-08      3.220001       0.286500  
        2022-02-09      2.670013      -0.263487  
        2023-02-16      6.989990       4.056489  

[14744 rows x 6 columns]

#### Monthly climatology method

Compute the monthly average climatology and use to compute anomaly.

Note: This method was not used in the final manuscript. It was calculated to compare sensitivity of outputs to the choice of climatology.

In [16]:
monthly_method = False

In [17]:
if monthly_method:
    # Calculate monthly climatologies
    geopolar_clim_monthly = geopolar_clim.groupby('time.month').mean()
    
    # Reshape climatology array so it matches shape of sst data
    list_of_arrays = []
    for time in geopolar.time:
        list_of_arrays.append(geopolar_clim_monthly.sel(month=time.dt.month))
    geopolar_clim_monthly = xr.concat(list_of_arrays, dim=geopolar.time)
    
    # Compute anomaly values
    geopolar_anom = geopolar - geopolar_clim_monthly

Timing Notes

Feb 1: ~22,000 rows: ~50 seconds


Find corresponding anomaly values

In [18]:
%%time

# Create a new column of the wq dataframe containing the corresponding geopolar sst value
# wq_anom['anom_geopolar'] = wq_anom.apply(lambda x: get_satellite_sst(
#     geopolar_monthlyanom, x.Latitude, x.Longitude, 
#     x.SampleDate), axis=1)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs


Find corresponding observed SST values

In [19]:
# Create a new column of the wq dataframe containing the corresponding geopolar sst value
wq_sst['geopolar'] = wq_sst.apply(lambda x: get_satellite_sst(geopolar, x.Latitude, x.Longitude, x.SampleDate), 
                                                axis=1)

In [20]:
# delete large data structures to save memory
# del geopolar

### MUR SST

In [21]:
# Open raw SST
path = (
    REPO_ROOT / 'data/01_raw' / 
    'MUR-JPL-L4_GHRSST-SSTfnd-GLOB-v02.0-fv04.1-CB-20030101_20231231.nc'
)
mur = xr.open_dataset(path).analysed_sst
# convert kelvin to celsius & update metadata
mur.values = mur.values - 273.15
mur.attrs.update({'units': 'celsius',})

# Open calculated climatological SST
# path = REPO_ROOT / 'data/interim/data_cache_Dec24' / 'mur_climatology_chesapeake.nc'
path = REPO_ROOT / 'data/02_interim' / 'mur_climatology_chesapeake.nc'
mur_clim = xr.open_dataset(path).climatology

#### Monthly climatology method

Compute the monthly average climatology and use to compute anomaly.

**Note:** This method was not used in the final manuscript. It was calculated to compare sensitivity of outputs to the choice of climatology.

In [22]:
monthly_method = False

In [23]:
if monthly_method:
    # Calculate monthly climatologies
    mur_clim_monthly = mur_clim.groupby('time.month').mean()
    
    # Reshape climatology array so it matches shape of sst data
    list_of_arrays = []
    for time in mur.time:
        list_of_arrays.append(mur_clim_monthly.sel(month=time.dt.month))
        mur_clim_monthly = xr.concat(list_of_arrays, dim=mur.time)
    
    # Compute anomaly values
    mur_anom = mur - mur_clim_monthly

Find corresponding anomaly values

In [24]:
# Create a new column of the wq dataframe containing the corresponding mur sst value
# wq_anom['anom_mur'] = wq_anom.apply(lambda x: get_satellite_sst(mur_anom, x.Latitude, x.Longitude, x.SampleDate), 
#                                                 axis=1)

#### Compute the custom climatology

Create monthly climatologies using only the dates which have corresponding observations from the CBP data.

In [25]:
%%time

satellite = 'mur'

anom_raw[f'clim_{satellite}'] = -999
anom_raw[f'sst_{satellite}'] = -999
for station in anom_raw.index.get_level_values(0).unique():
    print('station:', station)
    station_df = anom_raw.loc[station]
    # latitude and longitude should be the same for each row, so pick 0th value
    lat = station_df.Latitude[0]
    lon = station_df.Longitude[0]
    # loop through months
    for month in range(1, 13):
        # compile all dates from that month
        month_obs_dates = station_df[station_df.index.month == month].index
        # extract corresponding satellite values
        month_obs = []
        for date in month_obs_dates:
            sst = mur.sel(lat=lat, lon=lon, method='nearest') \
                .sel(time=date.strftime('%Y-%m-%d')).values
            sst = extract_matching_value(sst)
            # save sst observation value in the dataframe
            anom_raw.loc[(station, date), f'sst_{satellite}'] = sst
            # append value to array for climatology
            month_obs.append(sst)
        # average to make mean (--> ex. january climatological value)
        mean = np.nanmean(np.array(month_obs))
        # save sst_clim to a new row (subtract later)
        for date in month_obs_dates:
            anom_raw.loc[(station, date), f'clim_{satellite}'] = mean

station: CB1.1
station: CB2.1
station: CB2.2
station: CB3.1
station: CB3.2
station: CB3.3C
station: CB4.1C
station: CB4.2C
station: CB4.3C
station: CB4.4
station: CB5.1
station: CB5.2
station: CB5.3
station: CB6.2
station: CB6.3
station: CB6.4
station: CB7.3
station: CB7.3E
station: CB7.4
station: CB7.4N
station: CB8.1
station: CB8.1E
station: EE1.1
station: EE2.1
station: EE2.2
station: EE3.0
station: EE3.1
station: EE3.2
station: ET2.2
station: ET2.3


<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice


station: ET4.2
station: ET5.2
station: ET6.2
station: ET8.1
station: ET9.1
station: LE2.2
station: LE2.3
station: LE3.6
station: LE5.5-W


<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice


station: RET2.2
station: RET2.4
station: WE4.1
station: WE4.2
station: WE4.3
station: WE4.4
station: WT3.1
station: WT5.1
station: WT6.1
station: WT8.1


<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice


station: WT8.2
station: WT8.3
CPU times: user 31.8 s, sys: 911 ms, total: 32.7 s
Wall time: 33.2 s


<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty slice
<timed exec>:26: RuntimeWarning: Mean of empty

In [26]:
anom_raw['anom_mur'] = anom_raw['sst_mur'] - anom_raw['clim_mur']

In [27]:
anom_raw

Latitude  Longitude  anom_cbp  clim_geopolar  \
Station SampleDate                                                 
CB1.1   2003-01-15  39.54794  -76.08481    -2.950       3.399167   
        2005-01-13  39.54794  -76.08481     1.450       3.399167   
        2006-01-12  39.54794  -76.08481     0.950       3.399167   
        2007-01-12  39.54794  -76.08481     1.150       3.399167   
        2008-01-17  39.54794  -76.08481     0.650       3.399167   
...                      ...        ...       ...            ...   
WT8.3   2019-02-11  38.84250  -76.53410    -0.155       2.933501   
        2020-02-13  38.84250  -76.53410     2.945       2.933501   
        2021-02-08  38.84250  -76.53410    -0.755       2.933501   
        2022-02-09  38.84250  -76.53410    -0.905       2.933501   
        2023-02-16  38.84250  -76.53410     3.445       2.933501   

                    sst_geopolar  anom_geopolar  clim_mur   sst_mur  anom_mur  
Station SampleDate                                                             
CB1.1   2003-01-15      1.630005      -1.769162  3.876915  2.075989 -1.800926  
        2005-01-13      3.160004      -0.239164  3.876915  4.101013  0.224098  
        2006-01-12      3.910004       0.510836  3.876915  1.149994 -2.726921  
        2007-01-12      4.630005       1.230838  3.876915  6.183990  2.307076  
        2008-01-17      3.709991       0.310824  3.876915  3.701996 -0.174919  
...                          ...            ...       ...       ...       ...  
WT8.3   2019-02-11      2.260010      -0.673491       NaN       NaN       NaN  
        2020-02-13      3.980011       1.046510       NaN       NaN       NaN  
        2021-02-08      3.220001       0.286500       NaN       NaN       NaN  
        2022-02-09      2.670013      -0.263487       NaN       NaN       NaN  
        2023-02-16      6.989990       4.056489       NaN       NaN       NaN  

[14744 rows x 9 columns]

Find corresponding observed SST values

In [28]:
# Create a new column of the wq dataframe containing the corresponding mur sst value
wq_sst['mur'] = wq_sst.apply(lambda x: get_satellite_sst(mur, x.Latitude, x.Longitude, x.SampleDate), 
                                                axis=1)

In [29]:
# delete large data structures to save memory
# del mur

### OSTIA SST

Note: OSTIA has lots of missing dates because OSTIA begins in 2006, while the in situ records begin in 2003.

In [30]:
# Open raw SST
path = (
    REPO_ROOT / 'data/01_raw' / 
    'METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2_analysed_sst_77.47W-75.53W_36.78N-39.97N_2007-01-01-2023-12-31.nc'
)
ostia = xr.open_dataset(path).analysed_sst
ostia = ostia.rename({'latitude': 'lat', 'longitude': 'lon'})
# convert kelvin to celsius & update metadata
ostia.values = ostia.values - 273.15
ostia.attrs.update({'units': 'celsius',})

# Open calculated climatological SST
# path = REPO_ROOT / 'data/interim/data_cache_Dec24' / 'ostia_climatology_chesapeake.nc'
path = REPO_ROOT / 'data/02_interim' / 'ostia_climatology_chesapeake.nc'
ostia_clim = xr.open_dataset(path).climatology

#### Monthly climatology method

Compute the monthly average climatology and use to compute anomaly.

**Note:** This method was not used in the final manuscript. It was calculated to compare sensitivity of outputs to the choice of climatology.

In [31]:
monthly_method = False

In [32]:
if monthly_method:
    # Calculate monthly climatologies
    ostia_clim_monthly = ostia_clim.groupby('time.month').mean()
    
    # Reshape climatology array so it matches shape of sst data
    list_of_arrays = []
    for time in ostia.time:
        list_of_arrays.append(ostia_clim_monthly.sel(month=time.dt.month))
    ostia_clim_monthly = xr.concat(list_of_arrays, dim=ostia.time)
    
    # Compute anomaly values
    ostia_anom = ostia - ostia_clim_monthly

Find corresponding anomaly values

In [33]:
# Ensure all values are floats and none are np.ndarray
# wq_anom['anom_ostia'] = wq_anom['anom_ostia'].map(
#     lambda x: x.tolist() if isinstance(x, np.ndarray) else None
# )

In [34]:
# Create a new column of the wq dataframe containing the corresponding geopolar sst value
# wq_anom['anom_ostia'] = wq_anom.apply(
#     lambda x: get_satellite_sst(ostia_anom, x.Latitude, x.Longitude, 
#                                 x.SampleDate), axis=1)

#### Compute the custom climatology

Create monthly climatologies using only the dates which have corresponding observations from the CBP data.

In [35]:
%%time

satellite = 'ostia'

anom_raw[f'clim_{satellite}'] = -999
anom_raw[f'sst_{satellite}'] = -999
for station in anom_raw.index.get_level_values(0).unique():
    print('station:', station)
    station_df = anom_raw.loc[station]
    # latitude and longitude should be the same for each row, so pick 0th value
    lat = station_df.Latitude[0]
    lon = station_df.Longitude[0]
    # loop through months
    for month in range(1, 13):
        # compile all dates from that month
        month_obs_dates = station_df[station_df.index.month == month].index
        # extract corresponding satellite values
        month_obs = []
        for date in month_obs_dates:
            if date > pd.to_datetime('2006-12-31'):
                sst = ostia.sel(lat=lat, lon=lon, method='nearest') \
                    .sel(time=date.strftime('%Y-%m-%d')).values
                sst = extract_matching_value(sst)
                # save sst observation value in the dataframe
                anom_raw.loc[(station, date), f'sst_{satellite}'] = sst
                # append value to array for climatology
                month_obs.append(sst)
            else:
                anom_raw.loc[(station, date), f'sst_{satellite}'] = np.nan
        # average to make mean (--> ex. january climatological value)
        mean = np.nanmean(np.array(month_obs))
        # save sst_clim to a new row (subtract later)
        for date in month_obs_dates:
            anom_raw.loc[(station, date), f'clim_{satellite}'] = mean

station: CB1.1
station: CB2.1
station: CB2.2
station: CB3.1
station: CB3.2
station: CB3.3C
station: CB4.1C
station: CB4.2C
station: CB4.3C
station: CB4.4
station: CB5.1
station: CB5.2
station: CB5.3
station: CB6.2
station: CB6.3
station: CB6.4
station: CB7.3
station: CB7.3E
station: CB7.4
station: CB7.4N
station: CB8.1
station: CB8.1E
station: EE1.1
station: EE2.1
station: EE2.2
station: EE3.0
station: EE3.1
station: EE3.2
station: ET2.2


<timed exec>:29: RuntimeWarning: Mean of empty slice


station: ET2.3
station: ET4.2
station: ET5.2


<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice


station: ET6.2
station: ET8.1
station: ET9.1
station: LE2.2
station: LE2.3
station: LE3.6
station: LE5.5-W
station: RET2.2
station: RET2.4
station: WE4.1
station: WE4.2
station: WE4.3
station: WE4.4
station: WT3.1
station: WT5.1
station: WT6.1
station: WT8.1
station: WT8.2
station: WT8.3
CPU times: user 22.9 s, sys: 533 ms, total: 23.4 s
Wall time: 23.8 s


<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty slice
<timed exec>:29: RuntimeWarning: Mean of empty

In [36]:
anom_raw['anom_ostia'] = anom_raw['sst_ostia'] - anom_raw['clim_ostia']

In [37]:
anom_raw

Latitude  Longitude  anom_cbp  clim_geopolar  \
Station SampleDate                                                 
CB1.1   2003-01-15  39.54794  -76.08481    -2.950       3.399167   
        2005-01-13  39.54794  -76.08481     1.450       3.399167   
        2006-01-12  39.54794  -76.08481     0.950       3.399167   
        2007-01-12  39.54794  -76.08481     1.150       3.399167   
        2008-01-17  39.54794  -76.08481     0.650       3.399167   
...                      ...        ...       ...            ...   
WT8.3   2019-02-11  38.84250  -76.53410    -0.155       2.933501   
        2020-02-13  38.84250  -76.53410     2.945       2.933501   
        2021-02-08  38.84250  -76.53410    -0.755       2.933501   
        2022-02-09  38.84250  -76.53410    -0.905       2.933501   
        2023-02-16  38.84250  -76.53410     3.445       2.933501   

                    sst_geopolar  anom_geopolar  clim_mur   sst_mur  anom_mur  \
Station SampleDate                                                              
CB1.1   2003-01-15      1.630005      -1.769162  3.876915  2.075989 -1.800926   
        2005-01-13      3.160004      -0.239164  3.876915  4.101013  0.224098   
        2006-01-12      3.910004       0.510836  3.876915  1.149994 -2.726921   
        2007-01-12      4.630005       1.230838  3.876915  6.183990  2.307076   
        2008-01-17      3.709991       0.310824  3.876915  3.701996 -0.174919   
...                          ...            ...       ...       ...       ...   
WT8.3   2019-02-11      2.260010      -0.673491       NaN       NaN       NaN   
        2020-02-13      3.980011       1.046510       NaN       NaN       NaN   
        2021-02-08      3.220001       0.286500       NaN       NaN       NaN   
        2022-02-09      2.670013      -0.263487       NaN       NaN       NaN   
        2023-02-16      6.989990       4.056489       NaN       NaN       NaN   

                    clim_ostia  sst_ostia  anom_ostia  
Station SampleDate                                     
CB1.1   2003-01-15    3.614438        NaN         NaN  
        2005-01-13    3.614438        NaN         NaN  
        2006-01-12    3.614438        NaN         NaN  
        2007-01-12    3.614438   3.609985   -0.004452  
        2008-01-17    3.614438   3.739990    0.125553  
...                        ...        ...         ...  
WT8.3   2019-02-11    2.861764   2.290009   -0.571756  
        2020-02-13    2.861764   3.869995    1.008231  
        2021-02-08    2.861764   3.440002    0.578238  
        2022-02-09    2.861764   2.640015   -0.221750  
        2023-02-16    2.861764   7.440002    4.578238  

[14744 rows x 12 columns]

Find corresponding observed SST values

In [38]:
# Ensure all values are floats and none are np.ndarray
wq_sst['ostia'] = wq_sst['ostia'].map(
    lambda x: x.tolist() if isinstance(x, np.ndarray) else None
)

In [39]:
# Create a new column of the wq dataframe containing the corresponding geopolar sst value
wq_sst['ostia'] = wq_sst.apply(lambda x: get_satellite_sst(ostia, x.Latitude, x.Longitude, x.SampleDate), 
                                                axis=1)

In situ date not found in satellite SST 2006-08-07 at 38.64028 -77.22222
In situ date not found in satellite SST 2006-09-12 at 38.64028 -77.22222
In situ date not found in satellite SST 2006-10-10 at 38.64028 -77.22222
In situ date not found in satellite SST 2006-08-28 at 38.3475 -77.3275
In situ date not found in satellite SST 2006-04-11 at 38.4205 -77.3532
In situ date not found in satellite SST 2006-05-18 at 38.4205 -77.3532
In situ date not found in satellite SST 2006-06-15 at 38.4205 -77.3532
In situ date not found in satellite SST 2006-07-18 at 38.4205 -77.3532
In situ date not found in satellite SST 2006-08-09 at 38.4205 -77.3532
In situ date not found in satellite SST 2006-09-11 at 38.4205 -77.3532
In situ date not found in satellite SST 2006-10-25 at 38.4205 -77.3532
In situ date not found in satellite SST 2003-01-15 at 39.54794 -76.08481
In situ date not found in satellite SST 2003-03-12 at 39.54794 -76.08481
In situ date not found in satellite SST 2003-04-09 at 39.54794 -76.

In [40]:
wq_sst.ostia = wq_sst.ostia.astype('float')

In [ ]:
# delete large data structures to save memory
# del ostia

### Cleaning Output

Remove rows that don't have corresponding observations from any of the satellites.

In [41]:
anom_raw = anom_raw.reset_index()

In [42]:
wq_anom = anom_raw

In [43]:
wq_anom = wq_anom[(~wq_anom['anom_mur'].isnull()) | (~wq_anom['anom_geopolar'].isnull()) | \
                (~wq_anom['anom_ostia'].isnull())]

In [44]:
wq_sst = wq_sst[(~wq_sst['mur'].isnull()) | (~wq_sst['geopolar'].isnull())]

Sort values by date, latitude, and longitude

In [45]:
wq_anom = wq_anom.sort_values(['Station', 'SampleDate']).reset_index(drop=True)
wq_anom = wq_anom.round(decimals=4)

In [46]:
wq_sst = wq_sst.sort_values(['Station', 'SampleDate']).reset_index(drop=True)
wq_sst = wq_sst.round(decimals=4)

In [47]:
wq_anom

,Station,SampleDate,Latitude,Longitude,anom_cbp,clim_geopolar,sst_geopolar,anom_geopolar,clim_mur,sst_mur,anom_mur,clim_ostia,sst_ostia,anom_ostia
0,CB1.1,2003-01-15,39.5479,-76.0848,-2.9500,3.3992,1.63,-1.7692,3.8769,2.076,-1.8009,3.6144,NaN,NaN
1,CB1.1,2003-03-12,39.5479,-76.0848,-3.3500,4.4956,0.94,-3.5556,4.3557,-0.304,-4.6597,5.1571,NaN,NaN
2,CB1.1,2003-04-09,39.5479,-76.0848,-5.6200,10.5856,7.52,-3.0656,9.7780,4.105,-5.6729,11.0994,NaN,NaN
3,CB1.1,2003-04-23,39.5479,-76.0848,-0.1200,10.5856,9.19,-1.3956,9.7780,5.754,-4.0240,11.0994,NaN,NaN
4,CB1.1,2003-05-07,39.5479,-76.0848,-0.4321,14.2421,9.67,-4.5721,16.0494,9.337,-6.7124,15.1550,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14739,WT8.3,2023-08-08,38.8425,-76.5341,0.2083,26.4688,26.86,0.3912,NaN,NaN,NaN,26.8494,26.69,-0.1594
14740,WT8.3,2023-09-13,38.8425,-76.5341,4.0848,23.4165,26.58,3.1635,NaN,NaN,NaN,24.0041,26.21,2.2059
14741,WT8.3,2023-10-17,38.8425,-76.5341,-0.5042,18.0821,18.01,-0.0721,NaN,NaN,NaN,18.3747,18.12,-0.2547
14742,WT8.3,2023-11-16,38.8425,-76.5341,0.1381,12.5143,12.96,0.4457,NaN,NaN,NaN,12.8394,12.79,-0.0494


In [48]:
anom_raw

,Station,SampleDate,Latitude,Longitude,anom_cbp,clim_geopolar,sst_geopolar,anom_geopolar,clim_mur,sst_mur,anom_mur,clim_ostia,sst_ostia,anom_ostia
0,CB1.1,2003-01-15,39.54794,-76.08481,-2.950,3.399167,1.630005,-1.769162,3.876915,2.075989,-1.800926,3.614438,NaN,NaN
1,CB1.1,2005-01-13,39.54794,-76.08481,1.450,3.399167,3.160004,-0.239164,3.876915,4.101013,0.224098,3.614438,NaN,NaN
2,CB1.1,2006-01-12,39.54794,-76.08481,0.950,3.399167,3.910004,0.510836,3.876915,1.149994,-2.726921,3.614438,NaN,NaN
3,CB1.1,2007-01-12,39.54794,-76.08481,1.150,3.399167,4.630005,1.230838,3.876915,6.183990,2.307076,3.614438,3.609985,-0.004452
4,CB1.1,2008-01-17,39.54794,-76.08481,0.650,3.399167,3.709991,0.310824,3.876915,3.701996,-0.174919,3.614438,3.739990,0.125553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14739,WT8.3,2019-02-11,38.84250,-76.53410,-0.155,2.933501,2.260010,-0.673491,NaN,NaN,NaN,2.861764,2.290009,-0.571756
14740,WT8.3,2020-02-13,38.84250,-76.53410,2.945,2.933501,3.980011,1.046510,NaN,NaN,NaN,2.861764,3.869995,1.008231
14741,WT8.3,2021-02-08,38.84250,-76.53410,-0.755,2.933501,3.220001,0.286500,NaN,NaN,NaN,2.861764,3.440002,0.578238
14742,WT8.3,2022-02-09,38.84250,-76.53410,-0.905,2.933501,2.670013,-0.263487,NaN,NaN,NaN,2.861764,2.640015,-0.221750


Add a 0th row to each dataframe indicating column units

In [49]:
units_row = ['', '', 'degrees (epsg:4326)', 'degrees (epsg:4326)'] + ['deg C']*10
units_dict = dict(zip(wq_anom.columns, units_row))
units_df = pd.DataFrame(units_dict, index=[0])

wq_anom = pd.concat([units_df, wq_anom]).reset_index(drop=True)

In [50]:
units_row = ['', 'degrees (epsg:4326)', 'degrees (epsg:4326)', ''] + ['deg C']*4
units_dict = dict(zip(wq_sst.columns, units_row))
units_df = pd.DataFrame(units_dict, index=[0])

wq_sst = pd.concat([units_df, wq_sst]).reset_index(drop=True)

### Save File

In [53]:
path = (
    REPO_ROOT / 'data/03_processed' / 
    'SSTanom_satellites_cbp_stations.csv'
    # 'SSTanom_satellites_cbp_stations_Feb13(datespecific).csv'
)
wq_anom.to_csv(path, index=False)

In [54]:
wq_anom

,Station,SampleDate,Latitude,Longitude,anom_cbp,clim_geopolar,sst_geopolar,anom_geopolar,clim_mur,sst_mur,anom_mur,clim_ostia,sst_ostia,anom_ostia
0,,,degrees (epsg:4326),degrees (epsg:4326),deg C,deg C,deg C,deg C,deg C,deg C,deg C,deg C,deg C,deg C
1,CB1.1,2003-01-15 00:00:00,39.5479,-76.0848,-2.95,3.3992,1.63,-1.7692,3.8769,2.076,-1.8009,3.6144,NaN,NaN
2,CB1.1,2003-03-12 00:00:00,39.5479,-76.0848,-3.35,4.4956,0.94,-3.5556,4.3557,-0.304,-4.6597,5.1571,NaN,NaN
3,CB1.1,2003-04-09 00:00:00,39.5479,-76.0848,-5.62,10.5856,7.52,-3.0656,9.778,4.105,-5.6729,11.0994,NaN,NaN
4,CB1.1,2003-04-23 00:00:00,39.5479,-76.0848,-0.12,10.5856,9.19,-1.3956,9.778,5.754,-4.024,11.0994,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14740,WT8.3,2023-08-08 00:00:00,38.8425,-76.5341,0.2083,26.4688,26.86,0.3912,NaN,NaN,NaN,26.8494,26.69,-0.1594
14741,WT8.3,2023-09-13 00:00:00,38.8425,-76.5341,4.0848,23.4165,26.58,3.1635,NaN,NaN,NaN,24.0041,26.21,2.2059
14742,WT8.3,2023-10-17 00:00:00,38.8425,-76.5341,-0.5042,18.0821,18.01,-0.0721,NaN,NaN,NaN,18.3747,18.12,-0.2547
14743,WT8.3,2023-11-16 00:00:00,38.8425,-76.5341,0.1381,12.5143,12.96,0.4457,NaN,NaN,NaN,12.8394,12.79,-0.0494


In [56]:
path = (
    REPO_ROOT / 'data/03_processed' / 
    # 'SST_satellites_cbp_stations_Feb3(naivemonthly).csv'
    'SST_satellites_cbp_stations.csv'
)
wq_sst.to_csv(path, index=False)

In [59]:
wq_sst

,Station,Latitude,Longitude,SampleDate,sst_cbp,geopolar,mur,ostia
0,,degrees (epsg:4326),degrees (epsg:4326),,deg C,deg C,deg C,deg C
1,1AAUA001.39,38.4,-77.32,2007-03-22 00:00:00,8.3,4.0,NaN,2.97
2,1AAUA001.39,38.4,-77.32,2007-06-18 00:00:00,26.2,21.97,NaN,20.51
3,1AAUA001.39,38.4,-77.32,2007-10-29 00:00:00,14.5,18.02,NaN,17.01
4,1AAUA001.39,38.4,-77.32,2007-12-10 00:00:00,5.2,10.75,NaN,11.37
...,...,...,...,...,...,...,...,...
37275,YRK031.24,37.5046,-76.7925,2008-07-22 00:00:00,29.46,NaN,27.757,NaN
37276,YRK031.24,37.5046,-76.7925,2008-08-22 00:00:00,26.455,NaN,26.091,NaN
37277,YRK031.24,37.5046,-76.7925,2008-09-17 00:00:00,25.105,NaN,23.483,NaN
37278,YRK031.24,37.5046,-76.7925,2008-10-16 00:00:00,21.134,NaN,21.225,NaN
